# Lesson 05 — Building Your First A2A Agent

Build a standalone QA agent powered by **GitHub Phi-4** that answers questions about insurance policies.

## What You'll Learn

- Configure the OpenAI-compatible API for GitHub Models
- Load domain knowledge and inject it into a system prompt
- Build a `QAAgent` async class
- Test the agent standalone before adding A2A protocol layers

## Prerequisites

- GitHub account with a [Personal Access Token](https://github.com/settings/tokens) (no special scopes)
- `GITHUB_TOKEN` environment variable set
- Python 3.10+

> **GitHub Models docs:** [docs.github.com/en/github-models](https://docs.github.com/en/github-models)


## Step 1 — Install Dependencies


In [ ]:
# ── Dependencies ──────────────────────────────────────────────────────────────
# openai        → OpenAI-compatible SDK (used to call GitHub Models / Phi-4)
# python-dotenv → Loads GITHUB_TOKEN from the .env file automatically
#
# If you have the venv active (a2a-examples kernel selected) these are already
# installed. Run this cell anyway to ensure the kernel is up to date.
# ─────────────────────────────────────────────────────────────────────────────
%pip install openai python-dotenv --quiet
print("✅ Dependencies ready")

In [ ]:
import os
from dotenv import find_dotenv, load_dotenv

# ── Load secrets from .env ─────────────────────────────────────────────────
# find_dotenv() searches upward from this notebook's working directory.
# It will find _examples/.env which contains GITHUB_TOKEN.
#
# To set up your .env:
#   cp _examples/.env.example _examples/.env
#   # then edit .env and add:  GITHUB_TOKEN=ghp_your_token_here
#
# Get a free GitHub PAT (no special scopes needed):
#   https://github.com/settings/tokens
# ─────────────────────────────────────────────────────────────────────────────
env_path = find_dotenv(raise_error_if_not_found=False)
if env_path:
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path}")
else:
    print("⚠  .env not found — set GITHUB_TOKEN manually below if needed")
    # Uncomment and paste your token if .env is not available:
    # os.environ["GITHUB_TOKEN"] = "ghp_your_token_here"

token = os.environ.get("GITHUB_TOKEN", "")
if token:
    print(f"✅ GITHUB_TOKEN is set ({token[:8]}...)")
else:
    print("❌ GITHUB_TOKEN is NOT set — cells below will fail until you set it")

## Step 2 — Configure the Model Client

GitHub Models provides an **OpenAI-compatible API**. We use the standard `openai` package,
pointed at `https://models.inference.ai.azure.com` instead of OpenAI's endpoint.


In [ ]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI

# Load .env file if present (optional — you can also export GITHUB_TOKEN directly)
load_dotenv()

# Verify token is set
assert os.environ.get("GITHUB_TOKEN"), "Set GITHUB_TOKEN environment variable first!"

# Create the async OpenAI client pointing at GitHub Models
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

print(f"✅ Client configured — base_url: {client.base_url}")

## Step 3 — Quick Model Test

Before building the agent, verify that the model connection works.


In [ ]:
response = await client.chat.completions.create(
    model="Phi-4",
    messages=[{"role": "user", "content": "What is 2 + 2?"}],
    temperature=0.0,
)

print(f"Model: {response.model}")
print(f"Answer: {response.choices[0].message.content}")

## Step 4 — Load Domain Knowledge

The agent needs a knowledge base to answer domain-specific questions.
We load an insurance policy document and inject it into the system prompt.

This is **RAG-lite** — simple and effective for bounded domains.


In [ ]:
from pathlib import Path

SYSTEM_PROMPT = """\
You are a helpful insurance policy assistant.
Use the following policy document to answer questions accurately.
If the answer is not in the document, say so clearly.
Always cite the relevant section when possible.

--- POLICY DOCUMENT ---
{policy_text}
--- END DOCUMENT ---
"""


def load_knowledge(path: str) -> str:
    """Load a knowledge document from disk."""
    return Path(path).read_text(encoding="utf-8")


# Load the insurance policy
knowledge = load_knowledge("data/insurance_policy.txt")
system_prompt = SYSTEM_PROMPT.format(policy_text=knowledge)

print(f"📄 Loaded {len(knowledge)} characters of domain knowledge")
print(f"📝 System prompt: {len(system_prompt)} characters")

## Step 5 — Build the QAAgent Class

The agent encapsulates:

- Model client (AsyncOpenAI → GitHub Phi-4)
- Domain knowledge (loaded from file)
- System prompt (template with injected knowledge)

**Key design decisions:**
| Decision | Choice | Rationale |
|---|---|---|
| Async interface | `async def query()` | A2A servers are async — start async from day one |
| Class pattern | `QAAgent` class | Clean separation for AgentExecutor wrapping (Lesson 6) |
| Low temperature | `0.2` | Factual Q&A needs deterministic responses |


In [ ]:
class QAAgent:
    """Question-answering agent backed by GitHub Phi-4."""

    def __init__(
        self,
        knowledge_path: str,
        model: str = "Phi-4",
        temperature: float = 0.2,
    ):
        self.client = AsyncOpenAI(
            base_url="https://models.inference.ai.azure.com",
            api_key=os.environ["GITHUB_TOKEN"],
        )
        self.model = model
        self.temperature = temperature
        self.knowledge = load_knowledge(knowledge_path)
        self.system_prompt = SYSTEM_PROMPT.format(
            policy_text=self.knowledge
        )

    async def query(self, question: str) -> str:
        """Send a question to Phi-4 and return the answer."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": question},
            ],
            temperature=self.temperature,
        )
        return response.choices[0].message.content


print("✅ QAAgent class defined")

## Step 6 — Test the Agent

**Always test standalone before wrapping in A2A.** This verifies:

- Model connectivity (GitHub Models endpoint)
- Knowledge injection (system prompt)
- Response quality (grounded, factual answers)


In [ ]:
# Create the agent
agent = QAAgent("data/insurance_policy.txt")
print(f"✅ Agent created with {len(agent.knowledge)} chars of knowledge")

In [ ]:
# Test question 1: Specific fact
answer = await agent.query("What is the deductible for the Standard plan?")
print(f"❓ What is the deductible for the Standard plan?\n")
print(f"💬 {answer}")

In [ ]:
# Test question 2: Coverage question
answer = await agent.query("Are cosmetic procedures covered?")
print(f"❓ Are cosmetic procedures covered?\n")
print(f"💬 {answer}")

In [ ]:
# Test question 3: Out-of-scope question (should say "not in document")
answer = await agent.query("What is the capital of France?")
print(f"❓ What is the capital of France?\n")
print(f"💬 {answer}")

## Step 7 — Experiment

Try these variations:

- Change the `temperature` (0.0 for max determinism, 0.8 for more creativity)
- Ask follow-up questions about coverage limits, claims process, etc.
- Ask questions not covered by the document — the agent should say so clearly


In [ ]:
# Try your own question!
your_question = "How much is the monthly premium?"

answer = await agent.query(your_question)
print(f"❓ {your_question}\n")
print(f"💬 {answer}")

## Next Steps

This QAAgent is the **foundation** for every A2A agent in this course.

- **Lesson 6** → Wrap this agent as an A2A server (AgentExecutor + Agent Card)
- **Lesson 7** → Build a client that discovers and calls the server

The async class pattern — `QAAgent` with `async def query()` — is the core
that Lesson 6 wraps with the A2A SDK's `AgentExecutor` interface.
